In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.options.mode.chained_assignment = None

reviews_raw = pd.read_csv("rotten_tomatoes_movie_reviews.csv")
movies = pd.read_csv("rotten_tomatoes_movies.csv")


In [2]:
reviews_raw = reviews_raw.dropna(subset="originalScore")
movies = movies[["id", "title", "genre"]]


In [3]:
movies[movies.duplicated(keep=False, subset="id")].sort_values(by = "id")


,id,title,genre
134663,$5_a_day,Five Dollars a Day,Comedy
127459,$5_a_day,Five Dollars a Day,Comedy
129415,0s_and_1s,0s & 1s,Comedy
130221,0s_and_1s,0s & 1s,Comedy
129350,10004207-grand_slam,Grand Slam,Comedy
...,...,...,...
129438,zardoz,Zardoz,Sci-fi
129235,zavallilar,The Poor,Drama
129745,zavallilar,The Poor,Drama
9863,zombie_massacre_army_of_the_dead,Zombie Massacre: Army of the Dead,Horror


In [5]:
movies = movies.drop_duplicates("id")
reviews = pd.merge(reviews_raw, movies, on="id", how="left", validate="m:1")
reviews = reviews[["isTopCritic", "originalScore", "title", "genre"]]
reviews.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1009745 entries, 0 to 1009744
Data columns (total 4 columns):
 #   Column         Non-Null Count    Dtype 
---  ------         --------------    ----- 
 0   isTopCritic    1009745 non-null  bool  
 1   originalScore  1009745 non-null  object
 2   title          1005741 non-null  object
 3   genre          995216 non-null   object
dtypes: bool(1), object(3)
memory usage: 31.8+ MB


In [6]:
reviews["originalScore"].value_counts()


3/5               116711
4/5               110222
3/4                82297
2/5                72787
2/4                50645
                   ...  
5.9332482/10           1
8.701419/10            1
8.248120421/10         1
6.7/8                  1
7.44224848/10          1
Name: originalScore, Length: 1729, dtype: int64

In [7]:
reviews[reviews["originalScore"].str.contains("/")].info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 844782 entries, 0 to 1009744
Data columns (total 4 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   isTopCritic    844782 non-null  bool  
 1   originalScore  844782 non-null  object
 2   title          841462 non-null  object
 3   genre          832207 non-null  object
dtypes: bool(1), object(3)
memory usage: 26.6+ MB


In [56]:
reviews_std = reviews[reviews["originalScore"].str.contains("/")]
reviews_nonstd = reviews[~reviews["originalScore"].str.contains("/")]
reviews_nonstd["originalScore"].value_counts()


B                 30795
B+                24298
B-                20515
C                 16951
C+                15503
                  ...  
9.2                   1
7.8 out of 10         1
Aminus                1
0 out of 4..+4        1
5.8 out of 10         1
Name: originalScore, Length: 433, dtype: int64

In [57]:
reviews_std[["numerator", "denominator"]] = reviews_std.originalScore.str.split("/", expand=True, n=1)



In [58]:
reviews_std["num_clean"] = pd.to_numeric(reviews_std["numerator"], errors="coerce")
reviews_std["denom_clean"] = pd.to_numeric(reviews_std["denominator"], errors="coerce")



In [59]:
reviews_std[reviews_std["num_clean"].isna() | reviews_std["denom_clean"].isna()]


,isTopCritic,originalScore,title,genre,numerator,denominator,num_clean,denom_clean
8442,False,'3/4',Layer Cake,"Crime, Drama",'3,4',NaN,NaN
8475,False,'4/4',Layer Cake,"Crime, Drama",'4,4',NaN,NaN
10184,False,'3/4',Ferpect Crime,"Comedy, Drama, Crime, Mystery & thriller",'3,4',NaN,NaN
10196,False,'3/4',Ferpect Crime,"Comedy, Drama, Crime, Mystery & thriller",'3,4',NaN,NaN
10973,False,***1/2,A Place at the Table,Documentary,***1,2,NaN,2.0
...,...,...,...,...,...,...,...,...
1003666,False,'3.5/4',Broken Flowers,"Comedy, Drama",'3.5,4',NaN,NaN
1003694,False,'4/4',Broken Flowers,"Comedy, Drama",'4,4',NaN,NaN
1003726,False,'3.5/5',Broken Flowers,"Comedy, Drama",'3.5,5',NaN,NaN
1004758,False,'3/4',The Greatest Game Ever Played,Drama,'3,4',NaN,NaN


In [60]:
reviews_std = reviews_std[~reviews_std["denominator"].str.contains("/")]
reviews_std["numerator"] = reviews_std["numerator"].str.replace("[^0-9.]", "", regex=True)
reviews_std["denominator"] = reviews_std["denominator"].str.replace("[^0-9.]", "", regex=True)

reviews_std[reviews_std["num_clean"].isna() | reviews_std["denom_clean"].isna()]


,isTopCritic,originalScore,title,genre,numerator,denominator,num_clean,denom_clean
8442,False,'3/4',Layer Cake,"Crime, Drama",3,4,NaN,NaN
8475,False,'4/4',Layer Cake,"Crime, Drama",4,4,NaN,NaN
10184,False,'3/4',Ferpect Crime,"Comedy, Drama, Crime, Mystery & thriller",3,4,NaN,NaN
10196,False,'3/4',Ferpect Crime,"Comedy, Drama, Crime, Mystery & thriller",3,4,NaN,NaN
10973,False,***1/2,A Place at the Table,Documentary,1,2,NaN,2.0
...,...,...,...,...,...,...,...,...
1000141,False,2. 5 / 5,Scare Me,"Horror, Comedy",2.5,5,NaN,5.0
1003666,False,'3.5/4',Broken Flowers,"Comedy, Drama",3.5,4,NaN,NaN
1003694,False,'4/4',Broken Flowers,"Comedy, Drama",4,4,NaN,NaN
1003726,False,'3.5/5',Broken Flowers,"Comedy, Drama",3.5,5,NaN,NaN


In [62]:
reviews_std["num_clean"] = pd.to_numeric(reviews_std["numerator"], errors="coerce")
reviews_std["denom_clean"] = pd.to_numeric(reviews_std["denominator"], errors="coerce")

reviews_std[reviews_std["num_clean"].isna() | reviews_std["denom_clean"].isna()]

,isTopCritic,originalScore,title,genre,numerator,denominator,num_clean,denom_clean
240143,False,2.5/,Couples Retreat,Comedy,2.5,,2.5,NaN
371402,False,B-/C+,Team America: World Police,"Comedy, Action",,,NaN,NaN
408828,True,3.5/,The King of Kong: A Fistful of Quarters,"Documentary, Biography",3.5,,3.5,NaN
501657,False,1.5/,Insomnia,"Mystery & thriller, Crime, Drama",1.5,,1.5,NaN
703544,False,B/B-,Men in Black II,"Action, Comedy, Sci-fi",,,NaN,NaN
814454,False,2.5./4,The Time Traveler's Wife,"Romance, Drama, Sci-fi",2.5.,4,NaN,4.0
884030,False,3.5./4,Mein Führer: The Truly Truest Truth About Adol...,"Comedy, Drama",3.5.,4,NaN,4.0
886570,False,2/,The Situation,"Drama, War",2,,2.0,NaN
975956,False,4..5/5,The Business of Being Born,Documentary,4..5,5,NaN,5.0


In [63]:
reviews_std_clean = reviews_std[~(reviews_std["num_clean"].isna() | 
                            reviews_std["denom_clean"].isna() | reviews_std["denom_clean"] == 0)]
reviews_std_clean["decimal_score"] = reviews_std_clean["num_clean"] / reviews_std["denom_clean"]
reviews_std_clean.describe()

,num_clean,denom_clean,decimal_score
count,8.447530e+05,844752.000000,8.447490e+05
mean,1.586063e+01,6.201107,4.583673e+00
std,1.088016e+04,9.776181,3.626727e+03
min,0.000000e+00,0.500000,0.000000e+00
25%,2.500000e+00,4.000000,5.000000e-01
50%,3.000000e+00,5.000000,6.250000e-01
75%,4.000000e+00,5.000000,8.000000e-01
max,1.000000e+07,1000.000000,3.333334e+06


In [65]:
reviews_std_clean[reviews_std_clean["decimal_score"] > 1]

,isTopCritic,originalScore,title,genre,numerator,denominator,num_clean,denom_clean,decimal_score
12013,True,2.1/2,Don't Be Afraid of the Dark,"Fantasy, Horror, Mystery & thriller",2.1,2,2.1,2.0,1.05
17963,True,2.1/2,What to Expect When You're Expecting,"Comedy, Drama, Romance",2.1,2,2.1,2.0,1.05
20394,True,2.1/2,Green Lantern,"Action, Adventure, Sci-fi, Fantasy",2.1,2,2.1,2.0,1.05
27678,True,2.1/2,Everything Must Go,"Comedy, Drama",2.1,2,2.1,2.0,1.05
32589,True,2.1/2,Brighton Rock,"Crime, Drama, Mystery & thriller",2.1,2,2.1,2.0,1.05
...,...,...,...,...,...,...,...,...,...
977022,False,3 1/2 stars,The Green Hornet,"Action, Adventure",31,2,31.0,2.0,15.50
987151,True,2.1/2,Tower Heist,"Comedy, Adventure",2.1,2,2.1,2.0,1.05
992691,True,2.1/2,Rush,"Drama, Action",2.1,2,2.1,2.0,1.05
1002934,True,5.5/5,Vertigo,Mystery & thriller,5.5,5,5.5,5.0,1.10


These ratings do not make sense with the current decimal system, or do not specify the rating system being used, so they can be discarded. 

In [66]:
reviews_std_clean = reviews_std_clean[reviews_std_clean["decimal_score"] <= 1]
reviews_std_clean.describe()

,num_clean,denom_clean,decimal_score
count,844527.000000,844527.000000,844527.000000
mean,4.021020,6.201935,0.636915
std,7.157405,9.777252,0.207942
min,0.000000,1.000000,0.000000
25%,2.500000,4.000000,0.500000
50%,3.000000,5.000000,0.625000
75%,4.000000,5.000000,0.800000
max,100.000000,1000.000000,1.000000


Moving on to less standard rating configurations.

In [68]:
reviews_nonstd_frac = reviews_nonstd[reviews_nonstd["originalScore"].str.contains(" out of ")]

reviews_nonstd_frac[["num", "denom"]] = reviews_nonstd_frac.originalScore.str.split("out of", expand=True)

reviews_nonstd_frac["num_clean"] = pd.to_numeric(reviews_nonstd_frac["num"], errors="coerce")
reviews_nonstd_frac["denom_clean"] = pd.to_numeric(reviews_nonstd_frac["denom"], errors="coerce")

reviews_nonstd_frac[reviews_nonstd_frac["num_clean"].isna() | reviews_nonstd_frac["denom_clean"].isna()]


,isTopCritic,originalScore,title,genre,num,denom,num_clean,denom_clean
399,False,high +1 out of -4..+4,Falling Down,Action,high +1,-4..+4,NaN,NaN
645,False,high 0 out of -4..+4,Unstrung Heroes,Drama,high 0,-4..+4,NaN,NaN
4697,False,high +2 out of -4..+4,The Sixth Sense,Mystery & thriller,high +2,-4..+4,NaN,NaN
5839,False,0 out of -4..+4,Volcano,Action,0,-4..+4,0.0,NaN
7363,False,+1 out of -4..+4,The Serpent and the Rainbow,Horror,+1,-4..+4,1.0,NaN
...,...,...,...,...,...,...,...,...
990269,False,high +1 out of -4..+4,Thelma & Louise,"Comedy, Drama, Adventure",high +1,-4..+4,NaN,NaN
992205,False,+1 out of -4..+4,The Indian in the Cupboard,Fantasy,+1,-4..+4,1.0,NaN
997818,False,+1 out of -4..+4,Father of the Bride,Comedy,+1,-4..+4,1.0,NaN
998898,False,low +3 out of -4..+4,Leaving Las Vegas,Drama,low +3,-4..+4,NaN,NaN


 Let's assume that -4..+4 refers to a scale from -4 to +4. Therefore, in order to calculate the decimal rating, the numerator = numerator + 4, and every instance of -4..+4 can be changed to 8.

In [69]:
reviews_nonstd_frac["num"] = reviews_nonstd_frac["num"].str.replace("[^0-9.-]", "", regex=True)
reviews_nonstd_frac["num_clean"] = pd.to_numeric(reviews_nonstd_frac["num"], errors="coerce")
reviews_nonstd_frac.loc[reviews_nonstd_frac["denom"].str.contains("-4..+4"), "num_clean"] = reviews_nonstd_frac["num_clean"] + 4

reviews_nonstd_frac["denom"] = reviews_nonstd_frac["denom"].str.replace("-4..+4", "8", regex=False)
reviews_nonstd_frac["denom"] = reviews_nonstd_frac["denom"].str.replace("[^0-9.-]", "", regex=True)
reviews_nonstd_frac["denom_clean"] = pd.to_numeric(reviews_nonstd_frac["denom"], errors="coerce")

reviews_nonstd_frac[reviews_nonstd_frac["num_clean"].isna() | reviews_nonstd_frac["denom_clean"].isna()]


,isTopCritic,originalScore,title,genre,num,denom,num_clean,denom_clean
398590,False,0 out of 4..+4,Hellraiser,Horror,0,4..4,0.0,NaN


In [70]:
reviews_nonstd_frac_clean = reviews_nonstd_frac[~(reviews_nonstd_frac["num_clean"].isna() | 
                                                reviews_nonstd_frac["denom_clean"].isna())]

reviews_nonstd_frac_clean["decimal_score"] = reviews_nonstd_frac_clean["num_clean"] / reviews_nonstd_frac_clean["denom_clean"]
reviews_nonstd_frac_clean.describe()


,num_clean,denom_clean,decimal_score
count,803.000000,803.000000,803.000000
mean,5.255417,8.237858,0.637030
std,1.480018,1.040075,0.150353
min,0.000000,4.000000,0.000000
25%,4.000000,8.000000,0.545000
50%,5.000000,8.000000,0.625000
75%,6.000000,8.000000,0.750000
max,10.000000,10.000000,1.000000


In [71]:
reviews_nonstd = reviews_nonstd[~reviews_nonstd["originalScore"].str.contains(" out of ")]

reviews_nonstd["originalScore"].value_counts()


B         30795
B+        24298
B-        20515
C         16951
C+        15503
          ...  
60            1
9.2           1
Aminus        1
C+_           1
7.2           1
Name: originalScore, Length: 363, dtype: int64

The letter grades seem to operate on a scale from F, D, C, B, A, with B being the most common letter assignment. This implies a different rating-to-quality assignment from the standard 1-5 / 5 star rating, where the center element is most common. Conversions from these letter grades to a decimal are therefore risky and likely to not be properly comparable with the decimal ratings. 

In [78]:
reviews_cleaned = pd.concat([reviews_std_clean, reviews_nonstd_frac_clean])[["title", "isTopCritic", 
                                                                             "originalScore", "decimal_score", "genre"]]

reviews_cleaned.to_csv("professional_reviews_cleaned.csv")

In [79]:
reviews_cleaned.describe()

,decimal_score
count,845330.000000
mean,0.636915
std,0.207894
min,0.000000
25%,0.500000
50%,0.625000
75%,0.800000
max,1.000000


In [ ]:
reviews_std.describe()

